### Mongo Operation

In [4]:
from pymongo import MongoClient

from contextlib import contextmanager
import pandas as pd

@contextmanager
def connect_mongodb(username : str, password : str, host : str ,port : str):
    uri = f"mongodb://{username}:{password}@{host}:{port}"
    client  = None
    try:
        print("Connecting to MongoDB")
        client = MongoClient(uri)
        print("Connect to MongoDB")
        yield client
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if client:
            client.close()
            print("Stop Connect to MongoDB")
        else:
            print("Can't generate MongoDB Client")
        
class MongoDB_Operation:
    def __init__(self, client):
        """Initialize MongoDB connection."""
        if not isinstance(client, MongoClient):
            raise TypeError("client must be an instance of MongoClient")
        self.client = client

    def check_database_exists(self, db_name):
        """Check if a database exists."""
        return db_name in self.client.list_database_names()

    def check_collection_exists(self, db_name, collection_name):
        """Check if a collection exists in a database."""
        return collection_name in self.client[db_name].list_collection_names()

    def create_database(self, db_name):
        """Create a database if it does not exist."""
        if self.check_database_exists(db_name):
            print(f"Database '{db_name}' already exists!")
            return self.client[db_name]
        else:
            self.client[db_name].command("ping")
            print(f"Database '{db_name}' has been created successfully.")
            return self.client[db_name]
            
    def create_collection(self, db_name, collection_name):
        """Create a collection if it does not exist."""

        db = self.create_database(db_name)

        if self.check_collection_exists(db_name, collection_name):
            print(f"Collection '{collection_name}' already exists!")
            return db[collection_name]
        else:
            db.create_collection(collection_name)
            print(f"Collection '{collection_name}' has been created successfully.")
            return db[collection_name]

    def find_data(self, db_name, collection_name, query=None):
        """Find data in a collection."""
        if not isinstance(query, dict):
            raise TypeError("Query must be a dictionary")
        if not self.check_database_exists(db_name):
            print(f"Database '{db_name}' does not exist!")
        if not self.check_collection_exists(db_name, collection_name):
            print(f"Collection '{collection_name}' does not exist!")

        query = query or {}
        data = self.client[db_name][collection_name].find(query)
        return pd.DataFrame(data)

    def insert_many(self, db_name, collection_name, data : pd.DataFrame):
        """Insert multiple records into a collection."""
        if not isinstance(data, pd.DataFrame):
            raise TypeError("Data must be a Pandas DataFrame")
        
        coll = self.create_collection(db_name=db_name, collection_name=collection_name)

        data_dict = data.to_dict(orient='records')

        if data_dict:
            coll.insert_many(data_dict)
            print("Data has been added successfully.")
        else:
            print("Empty data, nothing to insert.")

In [5]:
import requests
import pandas as pd

API_KEY = "2d6e1b290dabf74f65b84431677db2b8"
GET_URL = "https://api.themoviedb.org/3/movie/popular"  # URL để lấy danh sách phim phổ biến

def get_movie_name(API_KEY, URL, pages = 10):
    
    list_movies = []  # Danh sách chứa các phim
    for page in range(1, pages + 1):

        # Tạo URL với API_KEY và số trang
        url = f'{URL}?api_key={API_KEY}&language=en-US&page={page}'
        response = requests.get(url)
        data = response.json()

        # Kiểm tra mã trạng thái của phản hồi
        if response.status_code == 200:
            results = data.get('results', [])
            for movie in results:
                title = movie.get('title')
                id = movie.get('id')
                if title:
                    list_movies.append([title, id])
        else:
            print(f"Error: {data.get('status_message', 'Unknown error')}")
            break
        
    print(f"Total movies: {len(list_movies)}")
    print("Getting Data Successfully!")
    return pd.DataFrame(list_movies, columns=['title', 'id'])

In [6]:
API_KEY = "2d6e1b290dabf74f65b84431677db2b8"
GET_URL = "https://api.themoviedb.org/3/movie/popular"

client = connect_mongodb(username="ndtien",
                         password="ndtien",
                         host="mongodb",
                         port="27017")

with client as mongo_client:
    db_operation = MongoDB_Operation(mongo_client)
    
    # Database and collection names
    db_name = "movies_db"
    collection_name = "movies_name"
    
    # Create database and collection
    db_operation.create_database(db_name)
    db_operation.create_collection(db_name, collection_name)
    
    # Insert data
    data = get_movie_name(API_KEY, GET_URL, pages = 20)
    db_operation.insert_many(db_name, collection_name, data)

Connecting to MongoDB
Connect to MongoDB
Database 'movies_db' already exists!
Database 'movies_db' already exists!
Collection 'movies_name' already exists!
Total movies: 400
Getting Data Successfully!
Database 'movies_db' already exists!
Collection 'movies_name' already exists!
Data has been added successfully.
Stop Connect to MongoDB
